In [1]:
from dotenv import load_dotenv
load_dotenv()
from IPython.display import display,Markdown
def md(text:str):
    return display(Markdown(text))

## 1. `Simple` Implementation using `Class Model`

In [2]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

In [3]:
# Step-1 : Model Setup
model = ChatGroq(model="qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000028EE297C980>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028EE297D6A0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

#### SystemMessage is okay here because the text is static (doesn't change)
    SystemMessage(content="You are a Helpful assistant"),
    
    MessagesPlaceholder(variable_name="history"),
    
    # You MUST use the PromptTemplate class to handle variables like {query}
    HumanMessagePromptTemplate.from_template("{query}")

#### ("role", "content") (recommaned)
    ("system", "You are a Helpful assistant"),
    
    # Placeholder for memory
    MessagesPlaceholder(variable_name="history"),
    
    # Use brackets {} to define a variable
    ("human", "{query}")

In [4]:
# 2. Create a prompt that expects history
from langchain.messages import SystemMessage,HumanMessage
prompt = ChatPromptTemplate.from_messages([
    # ("role", "content")
    ("system", "You are a Helpful assistant"),
    
    # Placeholder for memory
    MessagesPlaceholder(variable_name="history"),
    
    # Use brackets {} to define a variable
    ("human", "{query}")
])

In [5]:
# 3. Create the Chain (LCEL style)
chain = prompt | model

In [6]:
# 4. Setup Memory Management
# In a real app, 'store' would be a database (Redis, Postgres).
# For now, we use a simple dictionary.

store = {}
def get_session_history(session_id:str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [51]:
from rich import  print as pp
pp(store)

{
    'user_1': InMemoryChatMessageHistory(
        messages=[
            HumanMessage(content='hi my name is ash!', additional_kwargs={}, response_metadata={}),
            AIMessage(
                content='<think>\nOkay, the user introduced their name as Ash. I should acknowledge their greeting 
and respond in a friendly manner. Let me make sure to address them by name and offer assistance. I need to check 
for any additional details they might have mentioned, but since they just said "hi" and their name, a simple 
response should suffice. Let me ensure the tone is welcoming and open-ended to encourage them to ask questions or 
share more if they\'d like.\n</think>\n\nHi Ash! Nice to meet you! How can I assist you today? 😊',
                additional_kwargs={},
                response_metadata={
                    'token_usage': {
                        'completion_tokens': 112,
                        'prompt_tokens': 24,
                        'total_tokens': 136,
                        'completion_time': 0.243548685,
                        'completion_tokens_details': None,
                        'prompt_time': 0.001240531,
                        'prompt_tokens_details': None,
                        'queue_time': 0.069816249,
                        'total_time': 0.244789216
                    },
                    'model_name': 'qwen/qwen3-32b',
                    'system_fingerprint': 'fp_5cf921caa2',
                    'service_tier': 'on_demand',
                    'finish_reason': 'stop',
                    'logprobs': None,
                    'model_provider': 'groq'
                },
                id='lc_run--019bbd67-53e2-7d23-bee9-3abc2e00a835-0',
                tool_calls=[],
                invalid_tool_calls=[],
                usage_metadata={'input_tokens': 24, 'output_tokens': 112, 'total_tokens': 136}
            ),
            HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
            AIMessage(
                content='<think>\nAlright, the user just asked, "What is my name?" Let me check the conversation 
history.\n\nIn the previous messages, the user introduced themselves as "Ash" and I acknowledged that. Now they\'re
asking again. Maybe they want to confirm if I remember their name correctly. I should make sure to restate their 
name to show I\'m paying attention. It\'s possible they\'re testing if I retain information from the conversation. 
I should respond in a friendly and helpful manner, reaffirmating their name and offering further assistance. No 
need to overcomplicate it. Just a simple, polite reply should do.\n</think>\n\nYour name is Ash! 😊 I remember from
our previous message. How can I assist you today, Ash?',
                additional_kwargs={},
                response_metadata={
                    'token_usage': {
                        'completion_tokens': 151,
                        'prompt_tokens': 56,
                        'total_tokens': 207,
                        'completion_time': 0.34554086,
                        'completion_tokens_details': None,
                        'prompt_time': 0.002393572,
                        'prompt_tokens_details': None,
                        'queue_time': 0.053608542,
                        'total_time': 0.347934432
                    },
                    'model_name': 'qwen/qwen3-32b',
                    'system_fingerprint': 'fp_5cf921caa2',
                    'service_tier': 'on_demand',
                    'finish_reason': 'stop',
                    'logprobs': None,
                    'model_provider': 'groq'
                },
                id='lc_run--019bbd67-c687-72e0-b937-eef84fef7258-0',
                tool_calls=[],
                invalid_tool_calls=[],
                usage_metadata={'input_tokens': 56, 'output_tokens': 151, 'total_tokens': 207}
            ),
            HumanMessage(
                content='I am student of government college of 

In [8]:
# 5. Wrap the chain with Message History
chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="query",
    history_messages_key="history"
)
chain_with_memory

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x0000028EE3A1B740>, input_messages_key='query', history_messages_key='history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

- Usage of Model class

In [9]:
res1 = chain_with_memory.invoke(
    {
        "query":"hi my name is ash!"
    },
    config={
        "configurable":{
            "session_id":"user_1"
        }
    }
)

In [10]:
md(res1.content)

<think>
Okay, the user introduced their name as Ash. I should acknowledge their greeting and respond in a friendly manner. Let me make sure to address them by name and offer assistance. I need to check for any additional details they might have mentioned, but since they just said "hi" and their name, a simple response should suffice. Let me ensure the tone is welcoming and open-ended to encourage them to ask questions or share more if they'd like.
</think>

Hi Ash! Nice to meet you! How can I assist you today? 😊

In [11]:
res2 = chain_with_memory.invoke(
    {
        "query":"What is my name?"
    },
    config={
        "configurable":{
            "session_id":"user_1"
        }
    }
)

In [12]:
md(res2.content)

<think>
Alright, the user just asked, "What is my name?" Let me check the conversation history.

In the previous messages, the user introduced themselves as "Ash" and I acknowledged that. Now they're asking again. Maybe they want to confirm if I remember their name correctly. I should make sure to restate their name to show I'm paying attention. It's possible they're testing if I retain information from the conversation. I should respond in a friendly and helpful manner, reaffirmating their name and offering further assistance. No need to overcomplicate it. Just a simple, polite reply should do.
</think>

Your name is Ash! 😊 I remember from our previous message. How can I assist you today, Ash?

In [ ]:
res3 = chain_with_memory.invoke(
    {
        "query":"I am student of government college of engeniering , i have 2 girlfriends and 6 female friends . i am poor so i have sugar mommy to take care of my expenses , she is very nice to me never scolds me for anything and also teaches me maths and german language."
    },
    config={
        "configurable":{
            "session_id":"user_1"
        }
    }
)
md(res3.content)

<think>
Okay, the user introduced themselves as Ash, a student at a government engineering college. They mentioned having two girlfriends and six female friends, which might be a bit confusing because having two girlfriends implies they're in two romantic relationships, but having six female friends as well. Maybe it's a typo or misunderstanding, but I should note that.

Ash is poor and relies on a sugar mommy for expenses. The sugar mommy is described as very supportive, never scolding and helping with math and German. I need to be careful here. Sugar relationships can have complex dynamics, especially when there's a power imbalance. It's important to approach this with sensitivity without making assumptions about the nature of the relationship.

First, I should acknowledge the information provided. Then, consider possible areas where Ash might need support. Since they're a student, academic challenges could be an issue. Also, the financial situation might cause stress, and relying on someone else for support could have emotional implications. It's good that the sugar mommy is supportive, but it's worth gently exploring whether Ash feels comfortable in this arrangement or if there are any concerns.

I should avoid making judgments about the relationships. Instead, focus on offering help with studies (since the sugar mommy teaches math and German) or general advice on managing finances and relationships. Maybe ask if they need help with anything specific, like study tips, time management, or how to handle their finances in a way that empowers them long-term.

Also, considering the cultural context (Government College of Engineering), there might be local resources or student support services they could access. I should mention that as a possibility without assuming they're unaware of it.

Need to ensure the response is positive, encouraging, and non-judgmental. Encourage Ash to reach out if they need help with anything, whether it's academic, personal, or related to their financial situation. Emphasize that they're not alone and that there are ways to build a stable future beyond current arrangements.
</think>

Thank you for sharing about yourself, Ash! It sounds like you're navigating a complex but interesting chapter of life. Here’s a breakdown of how I can support you:

1. **Academic Support**:  
   Since you’re studying engineering and learning German, I’d be happy to help with **study tips**, **math problems**, or **German language practice**. Just let me know what you’re working on!

2. **Financial & Independence Goals**:  
   Relying on someone else for financial support can come with challenges. If you’re open to it, I can share **budgeting strategies**, **ways to build savings**, or ideas for **part-time work/student jobs** (like tutoring, freelancing, or campus jobs) to help you gain financial independence in the future.

3. **Relationship Dynamics**:  
   It’s great that your sugar mommy is supportive, but it’s also important to reflect on whether this arrangement aligns with your long-term goals. If you ever feel uncertain, I can help brainstorm **ways to set healthy boundaries** or explore relationships that empower you emotionally and financially.

4. **Social Life & Friendships**:  
   Balancing 2 girlfriends and 6 female friends sounds like a lot! If you’re ever curious about navigating relationships or social dynamics, I’m here to listen and offer perspective (no judgment, just honesty).

5. **Emotional Wellbeing**:  
   If you ever feel overwhelmed by your situations, I can help you process your feelings or suggest **stress-relief techniques** or **resources for mental health support**.

**How can I assist you today?** 😊  
- Need help with homework/study advice?  
- Want to brainstorm financial independence plans?  
- Looking for advice on relationships or friendships?  
- Just want to talk?  

You’re not alone in figuring this out, and I’m here to support you in a way that respects your goals and values. Let me know what’s on your mind! 🌟

In [14]:
res4 = chain_with_memory.invoke(
    {
        "query":"how many girlfriends and total female friends  do i have ? respond in numbers only like this gf -> X , total frnd -> Y"
    },
    config={
        "configurable":{
            "session_id":"user_1"
        }
    }
)
md(res4.content)

<think>
Okay, let's see. The user mentioned they have 2 girl friends and 6 female friends. Wait, the terms "girlfriends" and "female friends" might be a bit confusing here. The user is asking how many girlfriends and total female friends they have.

First, I need to clarify the terms. The user said "2 girl friends" and "6 female friends." But the term "girlfriends" usually refers to romantic partners, while "female friends" are just friends. However, in some contexts, people might use "girlfriends" to mean friends in general, but that's less common.

Wait, the user wrote "I have 2 girl friends and 6 female friends." So adding those together, 2 + 6 = 8. But the user is asking for the number of girlfriends (gf) and total female friends (total frnd). If "girlfriends" are the romantic partners here, then the 2 girl friends might be the romantic partners, and the 6 are just friends. But the user used the term "girl friends," which could be ambiguous. However, in the previous response, the assistant assumed that "girlfriends" are the romantic ones and "female friends" are friends. So in that case, the answer would be gf -> 2, total frnd -> 8.

But wait, the user wrote "I have 2 girl friends and 6 female friends." So if "girlfriends" are the romantic ones, then the 2 "girl friends" are the romantic partners, and the 6 are friends. Therefore, total female friends would be 2 (girlfriends) + 6 (friends) = 8. But the user is asking for "gf -> X , total frnd -> Y". So X is 2 (girlfriends) and Y is 8 (total female friends including the girlfriends).

But I need to make sure. Let me check the original message again. The user said "I have 2 girl friends and 6 female friends." So the 2 are "girl friends," which could be ambiguous. However, the assistant in the previous response interpreted "girlfriends" as romantic partners. So if the user meant 2 girlfriends and 6 female friends (non-romantic), then total female friends would be 2 + 6 = 8. Therefore, the answer should be gf -> 2, total frnd -> 8.

But maybe the user used "girl friends" to mean non-romantic friends. Then "girlfriends" in the question would refer to romantic partners, which the user hasn't mentioned. Wait, the user actually mentioned "I have 2 girl friends and 6 female friends," but didn't specify any romantic relationships. The assistant's previous response assumed that the 2 girl friends are girlfriends. However, the user might not have any girlfriends and just 2 girl friends and 6 female friends. So if the user is asking how many girlfriends and total female friends they have, but in their message they didn't mention any girlfriends, only friends, then the answer would be gf -> 0, total frnd -> 8. But that contradicts the assistant's previous assumption.

Wait, the user's initial message says "I have 2 girl friends and 6 female friends." The assistant's response mentioned "2 girlfriends" and "6 female friends," adding up to 8. So the assistant interpreted "girl friends" as "girlfriends." Therefore, the answer is gf -> 2, total frnd -> 8. But the user might not have any girlfriends. The confusion here is between "girl friends" (friends who are girls) and "girlfriends" (romantic partners). The user's message is a bit ambiguous. However, based on the previous assistant's interpretation, the answer would be.gf -> 2, total frnd -> 8. But the user might have intended "girl friends" as friends, not romantic partners. So in that case, the answer would be gf -> 0, total frnd -> 8. But the assistant's previous response assumed the 2 were girlfriends. Since the user is asking for numbers based on their own statement, the answer should reflect exactly what they said. If they said "2 girl friends and 6 female friends," and they are asking for "girlfriends" and "total female friends," then "girlfriends" would be 0 (if they are not romantic) and total female friends would be 2 + 6 = 8. But the assistant's previous message interpreted "girl friends" as "girlfriends." So there's a discrepancy here. To resolve this, the correct approach is to take the user's exact wording: "2 girl friends and 6 female friends." If "girlfriends" are separate from friends, then the user hasn't mentioned any girlfriends. Therefore, the answer would be gf -> 0, total frnd -> 8. However, the assistant previously assumed the 2 girl friends were girlfriends, but the user might not have intended that. So the accurate answer, based on the user's exact words, is gf -> 0, total frnd -> 8. But this could be a misunderstanding. Alternatively, if the user meant that the 2 are girlfriends and 6 are friends, then it's 2 + 6 = 8. But the user's wording is ambiguous. Since the user is asking for the numbers based on their own message, it's safer to take their words literally unless they clarify. So the answer should be gf -> 0, total frnd -> 8. However, the assistant's previous response might have interpreted "girl friends" as "girlfriends," leading to gf -> 2, total frnd -> 8. The user is now asking for clarification, so the assistant should respond based on the user's exact wording unless there's a clear indication otherwise. Therefore, the correct answer is gf -> 0, total frnd -> 8. But the assistant's previous message assumed 2 girlfriends, which might have been a mistake. To avoid confusion, the assistant should clarify. However, the user is now asking for the numbers based on their own message, so the assistant must respond accurately. Therefore, the correct numbers are gf -> 0, total frnd -> 8.
</think>

gf -> 0 , total frnd -> 8

## 2. `Simple` Implementation using `agent` (Requires LangGraph)

In [26]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain.messages import SystemMessage

# --- NEW IMPORTS FOR LANGGRAPH ---
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

In [27]:
# 1. SETUP THE MODEL
llm = ChatGroq(model="openai/gpt-oss-120b")

In [28]:
# 2. DEFINE A TOOL (Same as before)
@tool
def get_weather(location:str):
    """Returns the weather for a given location"""
    return f"its rainy in {location}."

tools_list = [get_weather]

In [29]:
# 3. MEMORY MANAGEMENT (The LangGraph Way)
# Instead of "RunnableWithMessageHistory", we use a "Checkpointer"
# This saves the state of the conversation automatically.
temp_memory = InMemorySaver()

In [37]:
# 4. CONSTRUCT THE AGENT
# In LangGraph, "create_react_agent" replaces "create_tool_calling_agent" + "AgentExecutor"
# "state_modifier" is where your System Prompt goes now.
system_prompt = "You are a helpful assistant with tools."

agent = create_react_agent(
    llm,
    tools_list,
    checkpointer=temp_memory,
    # messages_modifier=SystemMessage("You are very Helpful assistant."),
)

C:\Users\adars\AppData\Local\Temp\ipykernel_24832\451996678.py:6: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [39]:
import importlib.metadata
try:
    print("LangGraph Version:", importlib.metadata.version("langgraph"))
except importlib.metadata.PackageNotFoundError:
    print("LangGraph is not installed!")

LangGraph Version: 1.0.6


In [41]:
config = {"configurable": {"thread_id": "session_123"}}

In [42]:
response1 = agent.invoke(
    {"messages": [("human", "Hi, my name is Sebastian. The secret word is 'Apple'.")]}, 
    config=config
)

In [44]:
response1

{'messages': [HumanMessage(content="Hi, my name is Sebastian. The secret word is 'Apple'.", additional_kwargs={}, response_metadata={}, id='3b888653-b781-46b2-a01c-143f433854dd'),
  AIMessage(content='Hello Sebastian! Nice to meet you. I’ve noted your secret word. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says hi, introduces themselves, and gives a secret word "Apple". Likely they are establishing a secret word for authentication or something. We need to respond appropriately, acknowledging and maybe asking how we can help. There\'s no request for weather. So just respond politely, maybe ask what they need. Also note secret word may be used later. So respond acknowledging.'}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 135, 'total_tokens': 241, 'completion_time': 0.219254054, 'completion_tokens_details': {'reasoning_tokens': 75}, 'prompt_time': 0.006012089, 'prompt_tokens_details': None, 'queue_time': 0.054893061, 

In [45]:
md(response1['messages'][-1].content)

Hello Sebastian! Nice to meet you. I’ve noted your secret word. How can I assist you today?

In [49]:
response2 = agent.invoke(
    {"messages": [("human", "What is the length of the secret word?")]}, 
    config=config
)

In [50]:
md(response2['messages'][-1].content)

The secret word “Apple” has **5 characters**.